In [ ]:
# The usual preamble
%pylab inline

import pandas as pd
import numpy as np
import seaborn as sns; sns.set()

# Make the graphs a bit prettier, and bigger
#pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.family'] = 'sans-serif'

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

One of the main problems with messy data is: how do you know if it's messy or not?

We're going to use the NYC 311 service request dataset again here, since it's big and a bit unwieldy.

In [ ]:
requests = pd.read_csv('../data/311-service-requests.csv', compression='gzip')

# How do we know if it's messy? 

We're going to look at a few columns here. I know already that there are some problems with the zip code, so let's look at that first.
 
To get a sense for whether a column has problems, I usually use `.unique()` to look at all its values. If it's a numeric column, I'll instead plot a histogram to get a sense of the distribution.

When we look at the unique values in "Incident Zip", it quickly becomes clear that this is a mess.

Some of the problems:

* Some have been parsed as strings, and some as floats
* There are `nan`s 
* Some of the zip codes are `29616-0759` or `83`
* There are some N/A values that pandas didn't recognize, like 'N/A' and 'NO CLUE'

What we can do:

* Normalize 'N/A' and 'NO CLUE' into regular nan values
* Look at what's up with the 83, and decide what to do
* Make everything strings

In [ ]:
requests.head()

In [ ]:
#Show all Zips, grouped
requests["Incident Zip"].unique()

# Fixing the nan values and string/float confusion

We can pass a `na_values` option to `pd.read_csv` to clean this up a little bit. We can also specify that the type of Incident Zip is a string, not a float.

In [ ]:
# specifiy how to handle na_values and how to handle the Incidient Zip (In this case as a string)
na_values = ["NO CLUE", "N/A", "0"]
requests = pd.read_csv('../data/311-service-requests.csv', compression='gzip', na_values=na_values, dtype={'Incident Zip': str})

Display your output once again

In [ ]:
requests["Incident Zip"].unique()

# What's up with the dashes?

Get all rows with dashes

In [ ]:
# Write code to retrieve rows with dashes
rows_with_dashes = requests['Incident Zip'].str.contains('-').fillna(False)
len(requests[rows_with_dashes])

Display our rows with dashes

In [ ]:
# display rows with dashes
requests[rows_with_dashes]

I thought these were missing data and originally deleted them like this:

`requests['Incident Zip'][rows_with_dashes] = np.nan`

But then my friend Google pointed out that 9-digit zip codes are normal. Let's look at all the zip codes with more than 5 digits, make sure they're okay, and then truncate them.

In [ ]:
#show all zip codes greater than 5 / group them

long_zip_codes =requests['Incident Zip'].str.len() > 5

requests['Incident Zip'][long_zip_codes].unique()

Those all look okay to truncate to me.

In [ ]:
#truncate those zips, slice the first part and store it back to our incident zip column

requests['Incident Zip'] =  requests['Incident Zip'].str.slice(0,5)

Done.

Earlier I thought 00083 was a broken zip code, but turns out Central Park's zip code 00083! Shows what I know. I'm still concerned about the 00000 zip codes, though: let's look at that. 

In [ ]:
#Show all requests with zip starting 00000
requests[requests["Incident Zip"] == "00000"]

This looks bad to me. Let's set these to nan.

In [ ]:
#set all zips with 00000 to nan
zero_zips = requests['Incident Zip'] == '00000'
requests.loc[zero_zips, 'Incident Zip'] = np.nan

Great. Let's see where we are now:

In [ ]:
unique_zips = requests['Incident Zip'].unique()
unique_zips

Amazing! This is much cleaner. There's something a bit weird here, though -- I ask you to look up 77056 on Google maps?

Let's take a closer look:

In [ ]:
zips = requests['Incident Zip']
# Let's say the zips starting with '0' and '1' are okay, for now. (this isn't actually true -- 13221 is in Syracuse, and why?)
# is_close = zips that are close, starting with 0 and 1
is_close = zips.str.startswith('0') | zips.str.startswith('1')
# There are a bunch of NaNs, but we're not interested in them right now, so we'll say they're False
# is_far = not is close AND zips that are not null
is_far = ~(is_close) & zips.notnull()

In [ ]:
zips[is_far]
#display your output

In [ ]:
# show all far requests and their incident zip, descriptor and city. 
#Sort it by Incident Zip
requests[is_far][['Incident Zip', 'Descriptor','City']]

Okay, there really are requests coming from LA and Houston! Good to know. Filtering by zip code is probably a bad way to handle this -- we should really be looking at the city instead.

In [ ]:
#Filter all requesty by the city they're coming from
requests['City'].str.upper().value_counts()

It looks like these are legitimate complaints, so we'll just leave them alone.

# Putting it together

Here's what we ended up doing to clean up our zip codes, all together:

In [ ]:
na_values = ['NO CLUE', 'N/A', '0']
requests = pd.read_csv('../data/311-service-requests.csv', 
                       na_values=na_values, compression='gzip',
                       dtype={'Incident Zip': str})

In [ ]:
#write a function which sclices away the unnecesary part

def fix_zip_codes(zips):
    # Truncate everything to length 5 
    zips = zips.str.slice(0, 5)
    
    # Set 00000 zip codes to nan
    zero_zips = zips == '00000'
    zips[zero_zips] = np.nan
    
    return zips

In [ ]:
requests['Incident Zip'] = fix_zip_codes(requests['Incident Zip'])

In [ ]:
requests['Incident Zip'].unique()